# Using ChatGPT with Python

In [ ]:
!pip install openai

In this notebook you will learn how to use ChatGPT with Python for various tasks such as **text generation**, **speech-to-text**,  **text-to-speech**, and  **image generation**.

## Text Generation

OpenAI's text generation **models** (often called generative pre-trained transformers or large language models) have been trained to understand natural language, code, and images. The latest models, `gpt-4` and `gpt-3.5-turbo`, are accessed through the **chat completions API endpoint**.

> For an overview of the different models for text generation you can visit https://platform.openai.com/docs/guides/text-generation.

The models provide text outputs in response to  inputs given to them. The inputs to these models are also referred to as **"prompts"**. Designing a prompt is essentially how you “program” a large language model, usually by providing instructions or some examples of how to successfully complete a task.

To use one of these models via the OpenAI API, you can send a request containing the inputs and your API key, and receive a response containing the model's output.




### Chat Completions API

Chat models take a list of messages as input and return a model-generated message as output.

Before being able to use the Chat Completions API
endpoint we need to authenticate ourselves using the **API key**. You can find an API key which will work during this session in Canvas (in the course syllabus).



In [ ]:
import os
from pathlib import Path
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = 'COPY THE API_KEY HERE'

client = OpenAI()

After successful authentication the Chat Completions API can be used. The API call has two required inputs:

* `model`: the name of the model you want to use (e.g., `gpt-3.5-turbo`, `gpt-4`)
* `messages`: a list of message objects, where each object has two required fields:
   * `role`: the role of the messenger (either system, user, or assistant)
   * `content`: the content of the message (e.g., Write me a beautiful poem)

There are additional optional parameters:

* `temperature`: the temperature parameter allows you to balance randomness and determinism in the output. Low temperature values make the output more focussed and deterministic, while a high temperature increases randomness in the output and produce more unexpected outputs.

Typically, a conversation will start with a system message that tells the assistant how to behave, followed by alternating user and assistant messages, but you are not required to follow this format.

While the **system message** helps set the behavior of the assistant, the **user messages** provide requests or comments for the assistant to respond to, and  **assistant messages** store previous assistant responses, but can also be written by you to give examples of desired behavior.


An example Chat Completions API call:

In [ ]:
#Example with a system message
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Explain generative AI to a 6 year old."},

  ]
)

The response format of a Chat Completion API response:

* `id:` the ID of the request
* `object: `the type of object returned (e.g., chat.completion)
* `created:` the timestamp of the request
* `model:` the full name of the model used to generate the response
* `usage:` the number of tokens used to generate the replies, counting prompt, completion, and total
* `choices: `a list of completion objects (only one, unless you set n greater than 1)
* `message:` the message object generated by the model, with role and content
* `finish_reason:` the reason the model stopped generating text (either stop, or length if max_tokens limit was reached)
* `index: `the index of the completion in the list of choices

We can extract the reply as follows:

In [ ]:
print(response.choices[0].message.content)

### Some Prompting Tipps

The next section provides some prompting tipps. For more details you can visit [here](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_format_inputs_to_ChatGPT_models.ipynb).

#### System Messages

The system message can be used to prime the assistant with different personalities or behaviors.

System messages can be used for
*   defining roles
*   setting the tone
*   specifying the format



NOTE: Be aware that some models do not generally pay as much attention to the system message equally. For example, gpt-3.5-turbo-0301 does not generally pay as much attention to the system message as gpt-4-0314 or gpt-3.5-turbo-0613

In [ ]:
#System message defining the role
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a computer science teacher."},
    {"role": "user", "content": "Explain generative AI to a 6 year old."},
  ]
)

print(response.choices[0].message.content)

In [ ]:
#System message setting the tone
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a humorous AI. Your task is to write a fun, engaging article about hiking."},
  ]
)

print(response.choices[0].message.content)

In [ ]:
#System message specifying the format
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a computer science teacher. Provide 1) a list of advantages of using generative AI followed by 2) a list of disadvantages and risk."},
  ]
)

print(response.choices[0].message.content)

In [ ]:
#System message specifying the format
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a teacher teaching executive MBA students. 1) Explain what ML APIs such as Google Cloud Vision or Chat GPT are 2) Explain what open-source, off-the shelf pre-trained models are. 3) Compare the usage of ML APIs versus open-source, off-the shelf pretrained models."},
  ]
)

print(response.choices[0].message.content)

#### Few-shot Prompting

In some cases, it's easier to show the model what you want rather than tell the model what you want.

One way to show the model what you want is with faked example messages.

In [ ]:
# An example of a faked few-shot conversation to prime the model into translating business jargon to simpler speech
# Example adapted from: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_format_inputs_to_ChatGPT_models.ipynb

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)

To help clarify that the example messages are not part of a real conversation, and shouldn't be referred back to by the model, you can try setting the `name` field of `system` messages to `example_user` and `example_assistant`.


In [ ]:
# The business jargon translation example, but with example names for the example messages
# Example adapted from: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_format_inputs_to_ChatGPT_models.ipynb

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English."},
        {"role": "system", "name":"example_user", "content": "New synergies will help drive top-line growth."},
        {"role": "system", "name": "example_assistant", "content": "Things working well together will increase revenue."},
        {"role": "system", "name":"example_user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "system", "name": "example_assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)

In [ ]:
# description: classify titles into categories

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant that classifies titles into categories."},
        {"role": "system", "name": "example_user", "content": "Paris auf der Saslong nicht zu biegen"},
        {"role": "system", "name":"example_assistant", "content": "Ski Alpin"},
        {"role": "system", "name":"example_user", "content": "Kobayashi geht in Führung"},
        {"role": "system", "name":"example_assistant", "content": "Skispringen"},
        {"role": "system", "name": "example_user", "content": "Hütter fährt in Abfahrt aufs Podest"},
        {"role": "system", "name":"example_assistant", "content": "Ski Alpin"},
        {"role": "system", "name": "example_user", "content": "Schweizer Freudentag: Flury brilliert vor Hählen"},
        {"role": "system", "name":"example_assistant", "content": "Ski Alpin"},
        {"role": "system", "name": "example_user", "content": ",Arsenal unterliegt Tottenham im Derby"},
        {"role": "system", "name":"example_assistant", "content": "Fussball"},
        {"role": "user", "content": "Seoanes Gladbach muss sich mit Remis begnügen"},
        #{"role": "user", "content": "Superadler Kraft: Endlich einmal gelbe Weihnachten"},

    ],

    temperature=0,
)

print(response.choices[0].message.content)


### Using Chat Completions for Specific Tasks


#### Text Translation

In [ ]:
def translateFromEnglishToGerman(text):
  response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {
      "role": "system",
      "content": "You will be provided with a sentence in English, and your task is to translate it into German."
    },
    {
      "role": "user",
      "content": text
    }
  ],
  temperature=0.7,
  )
  return response

In [ ]:
response = translateFromEnglishToGerman("My name is Barbara. What is yours?")

In [ ]:
response.choices[0].message.content

#### Using ChatGPT for Image Prompt Generation

In [ ]:
def createImagePrompt(text):
  response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": text},
  ]
  )
  return response

In [ ]:
response = createImagePrompt("create 1 text to image prompt to create nice and cinematic mountain ranges")
generatedPrompt = response.choices[0].message.content
print(generatedPrompt)

## Text to Speech Generation

The Audio API provides a `speech` endpoint based on our TTS (text-to-speech) model. It comes with 6 built-in voices (`alloy`, `echo`, `fable`, `onyx`, `nova`, and `shimmer`) and can be used to:

* Narrate a written blog post
* Produce spoken audio in multiple languages
* Give real time audio output using streaming

To check out supported languages visit https://platform.openai.com/docs/guides/text-to-speech.

### Audio Generation Example

In [ ]:
def generateTextToSpeech(text):
  speech_file_path = "speech.mp3"
  response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=text
  )

  response.stream_to_file(speech_file_path)
  return speech_file_path

In [ ]:
import IPython.display as ipd
fileName=generateTextToSpeech("Today is a wonderful day to build something people love!")
ipd.Audio(filename=fileName)

In the next code snippet there is a translation step into the target language (e.g., German) before the audio output is generated.

In [ ]:
# Here we create a whole simple pipeline starting from translation and then speech generation

response = translateFromEnglishToGerman("My name is Barbara. What is yours?")
translatedText = response.choices[0].message.content

fileName=generateTextToSpeech(translatedText)
ipd.Audio(filename=fileName)

## Speech to Text (Audio API)

The Audio API provides two speech to text endpoints, `transcriptions` and `translations`, based on our state-of-the-art open source large-v2 Whisper model. They can be used to:

* Transcribe audio into whatever language the audio is in.
* Translate and transcribe the audio into english.

File uploads are currently limited to 25 MB and the
following input file types are supported: `mp3`, `mp4`, `mpeg`, `mpga`, `m4a`, `wav`, and `webm`.

### Transcription Example

In [ ]:
def transcribeAudio(fileName):
  audio_file = open(fileName, "rb")
  transcript = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    response_format="text"
  )
  return transcript

You will now be asked to upload an .mp3 file which will then be transcribed.

To explore the transcription capabilities of the Audio API you can experiment with one of the news audio files available at https://www.srf.ch/audio/nachrichten.

In [ ]:
!wget -O Nachrichten_21-01-2024-1600.1705850582243.mp3 "https://podcasts.srf.ch/world/audio/Nachrichten_21-01-2024-1600.1705850582243.mp3"

In [ ]:
# The file name which is passed as a parameter to the function transcribeAudio needs to correspond to the file you have uploaded
print(transcribeAudio("Nachrichten_21-01-2024-1600.1705850582243.mp3"))

Should German not be your mother tongue, then you can use the `translations` endpoint to transcribe the file into English.

In [ ]:
def transcribeAndTranslateAudio(fileName):
  audio_file = open(fileName, "rb")
  transcript = client.audio.translations.create(
    model="whisper-1",
    file=audio_file,
    response_format="text"
  )
  return transcript

In [ ]:
print(transcribeAndTranslateAudio("Nachrichten_21-01-2024-1600.1705850582243.mp3"))

### Working with Audio Segments

By default, the Whisper API only supports files that are less than 25 MB. To work with larger audio files, you will you will need to break them into chunks.

One option is to use the **PyDub open source Python package** to split the audio.

In [ ]:
!pip install pydub

In [ ]:
# code adapted from: https://platform.openai.com/docs/guides/speech-to-text/longer-inputs

from pydub import AudioSegment

def segmentAudio(audioFile):
  song = AudioSegment.from_mp3(audioFile)
  # PyDub handles time in milliseconds
  two_minutes = 2 * 60 * 1000
  first_2_minutes = song[:two_minutes]
  first_2_minutes.export("TwoMinutes" + audioFile, format="mp3")
  return "TwoMinutes" + audioFile

You will be asked to upload a file to be transcribed.

To explore the segmentation capabilities of the Audio API you can experiment with one longer audio files such as https://www.srf.ch/audio/tagesgespraech.

In [ ]:
!wget -O Tagesgespraech_18-01-2024-1300.1705581859927.mp3 "https://podcasts.srf.ch/world/audio/Tagesgespraech_18-01-2024-1300.1705581859927.mp3"

We can then play the split audio file.

In [ ]:
# The file name which is passed as a parameter to the function transcribeAudio needs to correspond to the file you have uploaded
fileName = "Tagesgespraech_18-01-2024-1300.1705581859927.mp3"
twoMinuteFile = segmentAudio(fileName)
ipd.Audio(filename=twoMinuteFile)

.. and even translate it to another language.

In [ ]:
print(transcribeAndTranslateAudio(twoMinuteFile))

To experiment a bit more with Whisper visit [here](https://github.com/openai/openai-cookbook/blob/main/examples/Whisper_processing_guide.ipynb). Moreover, if you want to learn more on prompting with Whisper visit [here](https://github.com/openai/openai-cookbook/blob/main/examples/Whisper_prompting_guide.ipynb).

## Image Generation



The Images API provides three methods for interacting with images:

* Creating images from scratch based on a text prompt (DALL·E 3 and DALL·E 2)
* Creating variations of an existing image (DALL·E 2 only)
* Creating edited versions of images by having the model replace some areas of a pre-existing image, based on a new text prompt (DALL·E 2 only)


The image generations endpoint allows you to create an original image given a text prompt. When using DALL·E 3, images can have a size of 1024x1024, 1024x1792 or 1792x1024 pixels.

By default, images are generated at `standard` quality, but when using DALL·E 3 you can set quality: "`hd`" for enhanced detail. Square, standard quality images are the fastest to generate.

You can request 1 image at a time with DALL·E 3 (request more by making parallel requests) or up to 10 images at a time using DALL·E 2 with the n parameter.

### Creating images based on a text prompt

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.images.generate(
  model="dall-e-3",
  prompt="a downhill skier in the swiss alps",
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url

In [ ]:
# Here we take the URL created previously and open the image
from PIL import Image
import urllib.request

with urllib.request.urlopen(image_url) as url:
    img=Image.open(url)
    display(img)

In [ ]:
# set a directory to save DALL·E images to
image_dir_name = "images"
image_dir = os.path.join(os.curdir, image_dir_name)

# create the directory if it doesn't yet exist
if not os.path.isdir(image_dir):
    os.mkdir(image_dir)

# print the directory to save to
#print(f"{image_dir=}")

In [ ]:
# save the image
import requests

def saveImageAs(generated_image_name):
  generated_image_filepath = os.path.join(image_dir, generated_image_name)
  generated_image_url = response.data[0].url  # extract image URL from response
  generated_image = requests.get(generated_image_url).content  # download the image

  with open(generated_image_filepath, "wb") as image_file:
    image_file.write(generated_image)  # write the image to the file

  return generated_image

In [ ]:
# any name you like; the filetype should be .png
generated_image = saveImageAs("generated_image.png")